In [ ]:
# Imports
import pandas as pd
import seaborn as sns
import historicdutchweather
import pytz
from typing import Tuple
import pvlib
from datetime import datetime
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# Business Understanding
We kijken naar zonnepaneeldata over een periode van net iets minder naar twee jaar. De metingen bestaan uit het wattage dat elke minuut door de panelen werd opgewekt. Hierbij is de data al omgezet naar watt per vierkante meter.

We zijn op zoek naar de efficientie van de panelen. Deze kan berekend worden door de theoretische hoeveelheid opbrengst te nemen en deze te verlagen met een bepaalde factor om de werkelijke meting te benaderen. Deze factor is de efficientie.

Om deze berekening te moeten doen, maken we gebruik van de pvlib library. Hierbij is het nodig om de helling van het zonnepaneel met het dak te weten (de tilt). En ook de richting van het paneel (de azimuth). De azimuth loopt van 0 graden noord, naar 90 graden oost, naar 180 graden zuid en dan naar 270 graden west.

# Data Understanding
## Stroomproductiedata ophalen

In [ ]:
df = pd.read_csv('dataset.csv')
df['time'] = pd.to_datetime(df['time'])

In [ ]:
# Doe hier je verder data verkenning

## Weerdataset ophalen

In [ ]:
# Maak gebruik van de historicdutchweather.get_local_weather functie of importeer rechtstreeks de data vanaf de KNMI site

## Data Preparation

## Resampling naar xx minuten

In [ ]:
# Resample je data naar 5 of 10 minuten

## Weer en zonnepanelen combineren

In [ ]:
# Combineer de zonnepaneel en weerdata
# Interpoleer de ontbrekende waarden, zodat de weerdata ook per xx minuten beschikbaar is

## Filter op zonnige dagen

# Modeling

In [ ]:
# Gegeven:
def model_zonnestraling(tilt:float, azimuth:float, tijdsreeks:pd.core.indexes.datetimes.DatetimeIndex, lat:float, lon:float) -> pd.DataFrame:
    """Berekent de hoeveelheid stroom die een paneel maximaal produceert met 100% efficientie"""
    
    zonne_positie_aan_hemel = pvlib.solarposition.get_solarposition(tijdsreeks, lat, lon)

    #locatie = pvlib.location.Location(latitude=lat, longitude=lon, name='Plaatsnaam', tz=pytz.timezone(r'Europe/Amsterdam'))
    locatie = pvlib.location.Location(latitude=lat, longitude=lon, name='Plaatsnaam', tz=pytz.timezone(r'UTC'))
    heldere_hemel = locatie.get_clearsky(tijdsreeks)  

    instraling = pvlib.irradiance.get_total_irradiance(tilt, 
                                                       azimuth, 
                                                       zonne_positie_aan_hemel['zenith'], 
                                                       zonne_positie_aan_hemel['azimuth'], 
                                                       heldere_hemel.dni, 
                                                       heldere_hemel.ghi, 
                                                       heldere_hemel.dhi)    
    
    return instraling['poa_global'].to_frame().rename(columns={'poa_global':'zonlicht'})

In [ ]:
# Aanvullen
def model_paneel(tilt:float, azimuth:float, efficientie:float, tijdsreeks:pd.core.indexes.datetimes.DatetimeIndex, lat:float, lon:float) -> pd.DataFrame:
    """Berekent de hoeveelheid geabsorbeerd zonlicht op een paneel op basis van de efficientie"""
    
    return ... # Aanvullen

In [ ]:
# Aanvullen
def loss(params:Tuple[float,float,float], df_waarneming:pd.DataFrame, lat:float, lon:float) -> float:
    
    (tilt, azimuth, efficientie) = params
    
    tijdsreeks = df_waarneming.index
    
    df_model = model_paneel(...) # Aanvullen
    
    return # Aanvullen
    

In [ ]:
# Eerste inschatting
tilt = 35   
azimuth = 180
efficientie = 0.2

In [ ]:
result = minimize(...) # Aanvullen
x = result['x']

## Evaluation

In [ ]:
print("Tilt:        {0:.1f}°".format(x[0]))
print("Azimuth:     {0:.1f}°".format(x[1]))
print("Efficientie: {0:.1f}%".format(x[2]*100))